In [1]:
import pandas as pd
from IPython.display import display
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

dir_suffix = "data/"
dir_suffix_entpackt = dir_suffix + "Daten_ausgepackt/"

In [2]:
print("Tip25W Testdaten Template:")
testdata = pd.read_csv(dir_suffix+"tip25W_testdaten_template.csv")

display(testdata.head(10))

print("Tip25W Trainingsdaten:")
trainingdata = pd.read_csv(dir_suffix+"tip25W_trainingsdaten.csv")
display(trainingdata.head(10))

print("Prior:")
df_orders_prior = pd.read_csv(dir_suffix_entpackt+ "order_products__prior.csv")
display(df_orders_prior.head(10))

print("orders")
df_orders = pd.read_csv(dir_suffix_entpackt + "orders.csv")
display(df_orders.head(10))

print("products:")
df_products = pd.read_csv(dir_suffix_entpackt + "products.csv")
display(df_products.head(10))

Tip25W Testdaten Template:


,Unnamed: 0,order_id,tip,tip_pc
0,10,1187899,NaN,NaN
1,25,1492625,NaN,NaN
2,49,2196797,NaN,NaN
3,74,525192,NaN,NaN
4,78,880375,NaN,NaN
5,82,1094988,NaN,NaN
6,88,1822501,NaN,NaN
7,115,1827621,NaN,NaN
8,129,2316178,NaN,NaN
9,200,2180313,NaN,NaN


Tip25W Trainingsdaten:


,Unnamed: 0,order_id,tip,tip_pc
0,0,2539329,True,0.092177
1,1,2398795,False,0.000000
2,2,473747,False,0.000000
3,3,2254736,True,0.102231
4,4,431534,True,0.092990
5,5,3367565,False,0.000000
6,6,550135,False,0.000000
7,7,3108588,False,0.000000
8,8,2295261,False,0.000000
9,9,2550362,False,0.000000


Prior:


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
5,2,17794,6,1
6,2,40141,7,1
7,2,1819,8,1
8,2,43668,9,0
9,3,33754,1,1


orders


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


products:


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
5,6,Dry Nose Oil,11,11
6,7,Pure Coconut Water With Orange,98,7
7,8,Cut Russet Potatoes Steam N' Mash,116,1
8,9,Light Strawberry Blueberry Yogurt,120,16
9,10,Sparkling Orange Juice & Prickly Pear Beverage,115,7


In [3]:

merged = trainingdata.merge(df_orders_prior, on="order_id", how="left")
merged = merged.merge(df_orders, on="order_id", how="left")
df_products["product_id"].is_unique
#order_merged = order_merged.merge(df_products, on="product_id", how="left")
display(merged.head(30))

,Unnamed: 0,order_id,tip,tip_pc,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,0,2539329,True,0.092177,196,1,0,1,prior,1,2,8,NaN
1,0,2539329,True,0.092177,14084,2,0,1,prior,1,2,8,NaN
2,0,2539329,True,0.092177,12427,3,0,1,prior,1,2,8,NaN
3,0,2539329,True,0.092177,26088,4,0,1,prior,1,2,8,NaN
4,0,2539329,True,0.092177,26405,5,0,1,prior,1,2,8,NaN
5,1,2398795,False,0.000000,196,1,1,1,prior,2,3,7,15.0
6,1,2398795,False,0.000000,10258,2,0,1,prior,2,3,7,15.0
7,1,2398795,False,0.000000,12427,3,1,1,prior,2,3,7,15.0
8,1,2398795,False,0.000000,13176,4,0,1,prior,2,3,7,15.0
9,1,2398795,False,0.000000,26088,5,1,1,prior,2,3,7,15.0


In [4]:
print(df_products["product_id"].is_unique)
df_products["product_id"].value_counts().head(10)
merged = merged.merge(df_products, on="product_id", how="left")

True


In [5]:
display(merged.head(1))

,Unnamed: 0,order_id,tip,tip_pc,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id
0,0,2539329,True,0.092177,196,1,0,1,prior,1,2,8,NaN,Soda,77,7


In [6]:
df_numeric =  merged[["add_to_cart_order", "reordered", "order_hour_of_day", "days_since_prior_order", "aisle_id", "department_id"]]
df_numeric.corr()
order_df = merged.groupby("order_id").agg(
    tip_pc=("tip_pc", "mean"),
    n_items=("product_id", "count"),
    mean_reordered=("reordered", "mean"),
    n_aisles=("aisle_id", "nunique"),
    n_departments=("department_id", "nunique"),
    order_number=("order_number", "first"),
    order_hour_of_day=("order_hour_of_day", "first"),
    order_dow=("order_dow", "first"),
    days_since_prior_order=("days_since_prior_order", "first"),
).reset_index()
order_df.head(10)

,order_id,tip_pc,n_items,mean_reordered,n_aisles,n_departments,order_number,order_hour_of_day,order_dow,days_since_prior_order
0,2,0.000000,9,0.666667,8,3,3,9,5,8.0
1,3,0.000000,8,1.000000,6,4,16,17,5,12.0
2,4,0.115910,13,0.923077,9,5,36,9,1,7.0
3,5,0.000000,26,0.807692,19,11,42,16,6,9.0
4,6,0.075219,3,0.000000,3,2,4,12,1,30.0
5,7,0.000000,2,0.000000,2,2,11,14,2,30.0
6,8,0.000000,1,1.000000,1,1,5,6,4,17.0
7,9,0.000000,15,0.666667,12,8,14,19,0,5.0
8,10,0.132824,15,0.533333,7,5,4,8,6,8.0
9,11,0.139572,5,1.000000,5,5,4,19,1,23.0


In [9]:
y = order_df["tip_pc"]

X = order_df.drop(columns=[
    "tip_pc",
    "order_dow",
    "order_id",
])
X["is_first_order"] = (X["order_number"] == 1).astype(int)
X["days_since_prior_order"] = X["days_since_prior_order"].fillna(0)
X.head(10)

,n_items,mean_reordered,n_aisles,n_departments,order_number,order_hour_of_day,days_since_prior_order,is_first_order
0,9,0.666667,8,3,3,9,8.0,0
1,8,1.000000,6,4,16,17,12.0,0
2,13,0.923077,9,5,36,9,7.0,0
3,26,0.807692,19,11,42,16,9.0,0
4,3,0.000000,3,2,4,12,30.0,0
5,2,0.000000,2,2,11,14,30.0,0
6,1,1.000000,1,1,5,6,17.0,0
7,15,0.666667,12,8,14,19,5.0,0
8,15,0.533333,7,5,4,8,8.0,0
9,5,1.000000,5,5,4,19,23.0,0


In [10]:
numeric_features = [
    "n_items",
    "mean_reordered",
    "n_aisles",
    "n_departments",
    "order_hour_of_day",
    "days_since_prior_order",
    "is_first_order",
    "order_number"
]
categorical_features = [
    "product_name"
]


preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features)
    ]
)

In [11]:

order_df.head(30)

,order_id,tip_pc,n_items,mean_reordered,n_aisles,n_departments,order_number,order_hour_of_day,order_dow,days_since_prior_order
0,2,0.000000,9,0.666667,8,3,3,9,5,8.0
1,3,0.000000,8,1.000000,6,4,16,17,5,12.0
2,4,0.115910,13,0.923077,9,5,36,9,1,7.0
3,5,0.000000,26,0.807692,19,11,42,16,6,9.0
4,6,0.075219,3,0.000000,3,2,4,12,1,30.0
5,7,0.000000,2,0.000000,2,2,11,14,2,30.0
6,8,0.000000,1,1.000000,1,1,5,6,4,17.0
7,9,0.000000,15,0.666667,12,8,14,19,0,5.0
8,10,0.132824,15,0.533333,7,5,4,8,6,8.0
9,11,0.139572,5,1.000000,5,5,4,19,1,23.0


In [12]:
model = LinearRegression()
pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("model", model)
    ]
)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("R²:", r2_score(y_test, y_pred))

MAE: 0.05906095794672046
RMSE: 0.06443116999766928
R²: 0.04039160161388644


In [ ]:
X.isna().sum().sort_values(ascending=False).head(20)


days_since_prior_order    2078068
add_to_cart_order               0
order_number                    0
reordered                       0
order_hour_of_day               0
product_name                    0
aisle_id                        0
department_id                   0
dtype: int64